In [12]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime, timedelta
import time
import random

In [26]:
dt=DynamoTable()
try:
    dt.select_table('LogFileScan')
    print(dt)
except:
    dt.create_table(
        table_name='LogFileScan',
        partition_key='PK',
        partition_key_type='S',
        rcu=50,
        wcu=50
    )

- Table name: LogFileScan            
- Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/LogFileScan            
- Table creation: 2023-09-02 21:16:38            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [1]:
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer


In [3]:
dynamodb_client = boto3.client('dynamodb')
# Serialize (Marshal) Example
item_to_store = {
    'PK': 'USER#12345',
    'user_id': '12345',
    'first_name': 'Terry',
    'age': 48,
}

serializer = TypeSerializer()

serialized_item = serializer.serialize(item_to_store)

put_item_params = {
    'TableName': 'LogFileScan',
    'Item': serialized_item['M'],
}

dynamodb_client.put_item(**put_item_params)

{'ResponseMetadata': {'RequestId': '5H6LHEECAFJGLHOFNO2B2PCFJRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 03 Sep 2023 00:28:56 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5H6LHEECAFJGLHOFNO2B2PCFJRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [22]:
get_item_params = {
    'TableName': 'LogFileScan',
    'Key': {
        'PK': {'S': 'USER#12345'},
    },
}

response = dynamodb_client.get_item(**get_item_params)

In [23]:
def dynamo_obj_to_python_obj(dynamo_obj: dict) -> dict:
    deserializer = TypeDeserializer()
    return {
        k: deserializer.deserialize(v) 
        for k, v in dynamo_obj.items()
    }  

In [27]:
json_data = dynamo_obj_to_python_obj(response['Item'])
json_data

{'user_id': '12345',
 'PK': 'USER#12345',
 'first_name': 'Terry',
 'age': Decimal('48')}

In [30]:
result = dt.query(pk_value="USER#12345")
result['Items']

Consumed Capacity: 0.5


{'user_id': '12345', 'PK': 'USER#12345', 'first_name': 'Terry', 'age': 48}